# S2W7D4: 诊断与调优 (Diagnosis & Optimization)
昨天我们给模型做了一次“体检”，发现它虽然及格了（Accuracy 60%），但是有点“偏科”（对稀有样本 Grab 不敢预测）。同时我们修改了代码，引入了 **Weighted Loss** 试图矫正这个问题。

今天，我们要化身为 **“AI 医生”**。我们不能只看分数的涨跌，我们要**看着病人的眼睛**（查看错误样本），找出它到底在哪里犯糊涂，然后通过**调参**（开药）来治好它。

**这是面试中“项目深挖”环节最容易被问到的地方**：“你遇到过 Bad Case 吗？你是怎么分析的？最后怎么解决的？”

## 1 🔍 核心任务：可视化“坏样本” (Bad Case Analysis)

Loss 只是一个数字，它不会告诉你模型为什么把“向左转”听成了“停止”。我们需要把预测错误的样本打印出来。

In [1]:
import sys
sys.path.append("../../") 
import torch
import numpy as np
import pandas as pd
from transformers import Trainer, TrainingArguments, BertTokenizer
from src.config import PRETRAINED_MODEL_DIR, CHECKPOINT_DIR
from src.models.model_bert import BertClassifier
from src.dataset.dataset import IntentDataset
from src.metrics import compute_metrics

# 1. 准备验证集 (这次多造一点难的数据，模拟真实场景)
val_texts = [
    "向左转", "停止", "抓起来", "请勿移动", "前进", 
    "把那个杯子拿给我", "紧急制动", "慢慢往前走", "向右转动一点点", "松开手"
]
# 假设 0:MOVE, 1:STOP, 2:GRAB
val_labels = [0, 1, 2, 1, 0, 2, 1, 0, 0, 2]
label_map = {0: "MOVE", 1: "STOP", 2: "GRAB"}

# 2. 加载昨天训练好的模型 (Checkpoint-24)
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_DIR)
val_dataset = IntentDataset(val_texts, val_labels, tokenizer)

model = BertClassifier(model_path=PRETRAINED_MODEL_DIR, num_labels=3)
# 修正加载方式，使用 safetensors
from safetensors.torch import load_file
# 请确认 checkpoint-24 是你昨天的最佳权重目录
best_ckpt = CHECKPOINT_DIR / "checkpoint-24" 
state_dict = load_file(f"{best_ckpt}/model.safetensors")
model.load_state_dict(state_dict)

# 3. 进行预测
training_args = TrainingArguments(output_dir="./results", per_device_eval_batch_size=4)
trainer = Trainer(model=model, args=training_args)
predictions = trainer.predict(val_dataset)

# 4. 提取结果
pred_logits = predictions.predictions
pred_ids = np.argmax(pred_logits, axis=-1)
true_ids = predictions.label_ids

# 5. ★★★ 核心：找出 Bad Cases ★★★
bad_cases = []
for i in range(len(val_texts)):
    if pred_ids[i] != true_ids[i]:
        bad_cases.append({
            "Text": val_texts[i],
            "True Label": label_map[true_ids[i]],
            "Pred Label": label_map[pred_ids[i]],
            "Confidence": np.max(torch.softmax(torch.tensor(pred_logits[i]), dim=-1).numpy())
        })

# 6. 用 Pandas 漂亮地打印出来
df = pd.DataFrame(bad_cases)
print("❌ 错误样本分析报告:")
if not df.empty:
    print(df.to_markdown())
else:
    print("🎉 太棒了！验证集全对！(说明该增加难度了)")

❌ 错误样本分析报告:
|    | Text             | True Label   | Pred Label   |   Confidence |
|---:|:-----------------|:-------------|:-------------|-------------:|
|  0 | 抓起来           | GRAB         | MOVE         |     0.474486 |
|  1 | 请勿移动         | STOP         | MOVE         |     0.420997 |
|  2 | 把那个杯子拿给我 | GRAB         | MOVE         |     0.462568 |
|  3 | 松开手           | GRAB         | MOVE         |     0.48105  |


**诊断思考 (Interview Point)**:

  * 如果模型把 "请勿移动" 预测成 `MOVE`：说明它只关注到了关键词 "移动"，没学到否定词 "请勿"。 -\> **对策**：增加带有否定词的训练数据。
  * 如果模型把 "抓起来" 预测成 `MOVE`：说明它还没学会 `GRAB` 这个生僻类别。 -\> **对策**：昨天加的 Weighted Loss 应该能缓解这个问题。

## 2 🎛️ 核心任务：启用 Weighted Loss 重训

昨天我们修改了 `model_bert.py`，但还没真正用它重新训练。今天我们要用带权重的 Loss 再跑一次，看看能不能把昨天 Precision=0 的尴尬解决掉。

### 步骤

1.  **重启 Kernel** (确保 `src.models.model_bert` 的修改生效)。
2.  **复制 Day 2 的训练代码** (或者直接在 Day 4 Notebook 里重写一遍 Trainer 流程)。
3.  **运行训练**。
4.  **再次评估**：重点看 `GRAB` 类别的 Recall 有没有上升。

**代码片段 (Day 4 Notebook):**

```python
# ... 定义 train_dataset (记得包含一些 Label=2 的样本) ...

# 重新初始化模型 (这时候它会读取我们在 init 里写好的 class_weights)
model_weighted = BertClassifier(model_path=PRETRAINED_MODEL_DIR, num_labels=3)

trainer = Trainer(
    model=model_weighted,
    args=training_args, # 稍微调大 epoch 试试，比如 5
    train_dataset=train_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
# 训练完立刻评估
print(trainer.evaluate())
```

In [3]:
import sys
sys.path.append("../../") 

import torch
import numpy as np
from transformers import Trainer, TrainingArguments, BertTokenizer
from src.config import PRETRAINED_MODEL_DIR, CHECKPOINT_DIR
from src.models.model_bert import BertClassifier
from src.dataset.dataset import IntentDataset
from src.metrics import compute_metrics

# -----------------------------------------------------------------------------
# 1. 构造“不平衡”的训练数据 (Imbalanced Data Construction)
# -----------------------------------------------------------------------------
# 场景模拟：
# Label 0 (MOVE): 12条 (占大多数，模型容易偷懒只猜这个)
# Label 1 (STOP): 3条  (稀有关键指令)
# Label 2 (GRAB): 3条  (稀有关键指令)
train_texts = [
    # --- 0: MOVE (大量) ---
    "向前走", "向后退", "左转", "右转", "前进十米", "后退五步", 
    "往左边移动", "往右边移动", "一直走", "慢速前进", "全速前进", "转个圈",
    # --- 1: STOP (少量) ---
    "停下", "紧急制动", "别动",
    # --- 2: GRAB (少量) ---
    "抓取杯子", "把苹果拿起来", "松开夹爪"
]
train_labels = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  # 12个 0
    1, 1, 1,                             # 3个 1
    2, 2, 2                              # 3个 2
]

# 验证集 (保持之前的即可，用来测试泛化能力)
val_texts = ["向左转", "停止", "抓起来", "请勿移动", "前进", "把那个杯子拿给我", "松开手"]
val_labels = [0, 1, 2, 1, 0, 2, 2]

print(f"训练集数量: {len(train_texts)}")
print(f"类别分布: MOVE={train_labels.count(0)}, STOP={train_labels.count(1)}, GRAB={train_labels.count(2)}")

# -----------------------------------------------------------------------------
# 2. 数据处理 (Data Pipeline)
# -----------------------------------------------------------------------------
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_DIR)
train_dataset = IntentDataset(train_texts, train_labels, tokenizer)
val_dataset = IntentDataset(val_texts, val_labels, tokenizer)

# -----------------------------------------------------------------------------
# 3. 初始化模型 (Model Initialization)
# -----------------------------------------------------------------------------
# ★★★ 关键点：这里初始化时，会调用你在 model_bert.py 里写的 class_weights ★★★
model_weighted = BertClassifier(model_path=PRETRAINED_MODEL_DIR, num_labels=3)
print("✅ 模型已加载，已启用 Weighted Loss (请确保 model_bert.py 已修改并保存)")

# -----------------------------------------------------------------------------
# 4. 配置训练参数 (Training Arguments)
# -----------------------------------------------------------------------------
training_args = TrainingArguments(
    output_dir="./results_weighted", # 换个输出目录，别覆盖之前的
    num_train_epochs=10,             # ★ 增加 Epoch，因为样本少，多学几轮让 Loss 收敛
    per_device_train_batch_size=4,   # Batch Size 稍微大一点点
    per_device_eval_batch_size=4,
    learning_rate=2e-5,              # 黄金学习率
    logging_steps=5,
    eval_strategy="epoch",     # 每个 epoch 测一次
    save_strategy="no",              # 实验阶段先不存 checkpoint，省空间
    no_cuda=False,
    seed=42
)

# -----------------------------------------------------------------------------
# 5. 训练与评估 (Train & Evaluate)
# -----------------------------------------------------------------------------
trainer = Trainer(
    model=model_weighted,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print("\n🚀 开始 Weighted Loss 训练...")
trainer.train()

print("\n📊 最终评估结果:")
metrics = trainer.evaluate()
print(metrics)

训练集数量: 18
类别分布: MOVE=12, STOP=3, GRAB=3
✅ 模型已加载，已启用 Weighted Loss (请确保 model_bert.py 已修改并保存)

🚀 开始 Weighted Loss 训练...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.147100,1.072649,0.571429,0.448980,0.400000,0.571429
2,0.910500,1.064112,0.428571,0.306122,0.257143,0.428571
3,0.736300,1.034247,0.428571,0.306122,0.257143,0.428571
4,0.623100,0.926951,0.428571,0.304762,0.238095,0.428571
5,0.446800,0.838171,0.571429,0.557143,0.714286,0.571429
6,0.391500,0.791867,0.714286,0.714286,0.761905,0.714286
7,0.325300,0.741361,0.714286,0.714286,0.761905,0.714286
8,0.235700,0.709983,0.714286,0.714286,0.761905,0.714286
9,0.298700,0.693440,0.714286,0.714286,0.761905,0.714286
10,0.234600,0.686686,0.714286,0.714286,0.761905,0.714286


/home/goodminton/anaconda3/envs/utils/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/goodminton/anaconda3/envs/utils/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/goodminton/anaconda3/envs/utils/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"


📊 最终评估结果:


{'eval_loss': 0.6866857409477234, 'eval_accuracy': 0.7142857142857143, 'eval_f1': 0.7142857142857143, 'eval_precision': 0.7619047619047619, 'eval_recall': 0.7142857142857143, 'eval_runtime': 0.0924, 'eval_samples_per_second': 75.742, 'eval_steps_per_second': 21.641, 'epoch': 10.0}


## 3 🧪 进阶任务：超参数调优 (Hyperparameter Tuning)

算法工程师的日常就是“炼丹”。你需要手动调整 `TrainingArguments` 里的参数，感受它们对结果的影响。

请尝试修改以下参数，并记录 Accuracy 的变化：

| 参数 | 建议尝试值 | 预期影响 |
| :--- | :--- | :--- |
| **Learning Rate** | `5e-5` (变大) | 收敛变快，但可能震荡不收敛 (Loss 乱跳)。 |
| **Learning Rate** | `1e-5` (变小) | 收敛极慢，需要更多 Epoch，但结果可能更精细。 |
| **Batch Size** | `4` 或 `8` | 梯度更稳定，但显存占用增加。 |
| **Num Epochs** | `5` 或 `10` | 训练更久，小心过拟合 (Val Loss 升高)。 |

**面试必问**: *“你在这个项目中使用了什么 Learning Rate？为什么？”*
**标准答案**: *“我使用了 BERT 微调的黄金区间 2e-5。我尝试过 5e-5，发现 Loss 震荡比较大；尝试过 1e-5，收敛太慢。2e-5 是一个比较好的平衡点。”*

## 4 ⚔️ 每日算法题 (LeetCode 8)

既然我们在处理 NLP 意图识别（把字符串变成指令），今天我们来做一道极其经典的**字符串解析 (String Parsing)** 题目。这道题考察的是你处理“脏数据”和边界条件的细心程度，这在 NLP 预处理中非常重要。

### 🎯 题目: [LeetCode 8. 字符串转换整数 (atoi)](../../LeetCode%20practice/1-50.ipynb)

  * **难度**: Medium
  * **标签**: 字符串
  * **描述**: 实现 `myAtoi(string s)` 函数，将字符串转换成一个 32 位有符号整数。
      * **步骤**:
        1.  **丢弃前导空格**。
        2.  **检查符号** (`+` 或 `-`)。
        3.  **读入数字**：直到到达非数字字符或结尾。
        4.  **截断**：如果超过 32 位整数范围 $[-2^{31}, 2^{31}-1]$，则截断。
      * **示例**: `"   -42 with words"` -\> `-42`
  * **为什么选这题**: 你的“指令识别”项目中，如果用户输入 "向左转 30 度"，你需要解析出 `30` 这个参数。这道题就是参数解析的简化版。

## ✅ Day 4 任务清单

1.  **坏样本可视化**: 运行代码，打印出 Pandas 表格，亲眼看看模型错在哪。
2.  **验证 Weighted Loss**: 重新训练模型，观察 `eval_recall` 是否提升，确认模型是否敢于预测 `GRAB` 了。
3.  **手动调参**: 至少尝试修改一次 Learning Rate，并记录结果对比。
4.  **算法**: 通过 LeetCode 8。

准备好了吗？让我们开始诊断！请回复 **“Start Day 4”**。